이전 단계에서 생성한 incidents_test_minedHN.jsonl 파일을  
모델 테스트 데이터 포맷으로 변환 및 저장.  

incidents_ft_minedHN.jsonl은 그대로 사용가능하므로 냅둔다.


In [ ]:
from datasets import load_dataset, Features, Value

fpath = "../dataset/preprocessing/incidents_test_minedHN.jsonl"
ds = load_dataset('json', data_files=fpath)['train']

# 새 피처 정의
new_features = Features({
    'id': Value('string'),
    'title': ds.features['title'],
    'query': ds.features['query'],
    'pos': ds.features['pos'],
    'neg': ds.features['neg'],
})

# ds = ds(new_features)


def int_to_str(row):
    row['id'] = str(row['id'])
    return row

ds = ds.map(features=new_features)
# ds = ds.map(int_to_str, features=new_features)
ds[0]

In [ ]:
queries = ds.select_columns(['id', 'query'])
queries = queries.rename_column('query', 'text')

corpus = ds.select_columns(['id', 'pos'])
corpus = corpus.rename_column('pos', 'text')

qrels = ds.select_columns(['id'])
qrels = qrels.rename_column('id', 'qid')
qrels = qrels.add_column('docid', ds['id'])
qrels = qrels.add_column('relevance', [1] * len(ds)) # 연관도 수치는 1로 고정

In [ ]:
queries[1]

In [ ]:
corpus[1]

In [ ]:
qrels[1]

In [ ]:
queries.to_json("../dataset/test/gpt_test_queries.jsonl", force_ascii=False)
corpus.to_json("../dataset/test/gpt_test_corpus.jsonl", force_ascii=False)
qrels.to_json("../dataset/test/gpt_test_qrels.jsonl", force_ascii=False)